# Load Libraries

In [1]:
import os
import pandas as pd
import numpy as np
# from d2l import torch as d2l
# from torch import nn
from sklearn.model_selection import train_test_split
import csv
# import torch

# Load Dataset

In [2]:
kaggle_data = pd.read_json('train.json')
kaggle_data.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


# Split the dataset into train and set

In [3]:
# Split the first 90% of the data as the train set
test_size = round(len(kaggle_data)*0.1) 
train_size = len(kaggle_data) - test_size

# train_set, test_set = train_test_split(kaggle_data, test_size=len(kaggle_data)-train_size, random_state = False)
train_set = kaggle_data.iloc[:train_size].copy()
test_set = kaggle_data.iloc[train_size:].copy()

# train_set
test_size

train_set.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


# Word embedding with BERT

Load the libraries

In [30]:
import torch
from transformers import BertTokenizer, BertModel
import logging
import matplotlib.pyplot as plt
import nltk
%matplotlib inline
pd.set_option('display.max_colwidth', None)


Load the Bert Tokenizer and NLTK to split essays into individual sentences

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [32]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sblan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Define a function to split given text into an array of sentences

In [33]:
def split_to_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

test_set.head(5)['sentences']

6126                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [41]:
# Create a new column that has the the sentence splits
test_set['sentences'] = test_set['full_text'].apply(split_to_sentences)

In [43]:
input_ids = []
attention_masks = []

# Iterate over each row in the DataFrame
for index, row in test_set.iterrows():
    sentences = row['sentences']
    
    # For each sentence in the 'sentences' column...
    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sentence,                  
                            add_special_tokens=True,   # Add '[CLS]' and '[SEP]'
                            max_length=100,             # Adjust sentence length
                            pad_to_max_length=True,    # Pad/truncate sentences
                            return_attention_mask=True,# Generate attention masks
                            return_tensors='pt',       # Return PyTorch tensors
                       )

        input_ids.append(encoded_dict['input_ids'])

        # Construct an attention mask (identifying padding/non-padding).
        attention_masks.append(encoded_dict['attention_mask'])


In [44]:
encoded_dic

{'input_ids': tensor([[  101,  2750,  2023,  1010,  2178,  9398,  6994,  2008,  2071,  4906,
          1999,  2023,  7551,  2003,  5107,  3989,  1010,  2073,  4620,  1997,
          1037,  3144,  3208,  2030,  1037,  3407,  2155,  2052,  2022,  3491,
          1010,  1999,  2344,  2000,  2156,  2029,  2765,  2062,  8702,  2005,
          1996,  5347,  1998, 15306,  2052,  2022,  4567,  2011,  5517,  1996,
          5876, 12515,  2092, 19205,  3070,  3855,  2077,  1010,  5052,  1996,
          4500,  2344,  1997,  1996,  2832,  4541,  2682,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0